# IWT Product Type Proportion Charts
### Code reads in a specifically formatted Excel file, and creates a chart based on the proportions of IWT products encountered in the literature review, per species.

In [8]:
import pandas as pd

# Load the Excel file
df = pd.read_excel("Data/Book2.xlsx")

# Fix necessary column text
df["species"] = df["species"].astype(str).str.strip().str.lower()
df["animal product type"] = df["animal product type"].astype(str).str.strip()
df["label"] = df["label"].astype(str).str.strip()

# Remove missing product types
df = df[df["animal product type"] != "not specified"]

# Get unique species-product combinations per label
df_unique = df.drop_duplicates(subset=["label", "species", "animal product type"])

# Count the species-product combinations
product_type_counts = df_unique.groupby(["species", "animal product type"]).size().reset_index(name="count")

# Fix species names
product_type_counts["species"] = product_type_counts["species"].str.capitalize()

# Change product category labels
product_type_counts["animal product type"] = product_type_counts["animal product type"].replace({"whole":"Live Animal", "derivative":"Dead Animal/Animal Derivative", "not specified":"Unknown"})

# Export data
product_type_counts.to_excel("Output/AnimalProductSummary.xlsx", index=False)


In [7]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.font_manager import FontProperties

# Set global font
rcParams["font.family"] = "Century Gothic"

# Set colours for each product category
custom_colours = {
    "Live Animal": "#09899b",
    "Dead Animal/Animal Derivative": "#765710"
}

# Per species
for species in product_type_counts["species"].unique():
        
    # Get only the relevant data for that species
    species_df = product_type_counts[product_type_counts["species"] == species].copy()
    
    # Convert counts into percentages
    total_count = species_df["count"].sum()
    species_df.loc[:, "Percent"] = ((species_df["count"] / total_count) * 100)
    
    # Create the plot
    fig, ax = plt.subplots(figsize=(5, 6), facecolor="white")
    bottom = None
    
    # Get product types
    product_types = species_df["animal product type"].unique()
    
    # Create manual legend to follow the same order as the bar chart
    legend_handles = []
    legend_labels = []
    
    # For each product type
    for product_type in product_types:
        
        # Get relevant data
        product_data = species_df[species_df["animal product type"] == product_type]
        
        # Get proportion
        proportion = product_data["Percent"].values
        
        # Create the bar
        ax.bar(
            [species],
            proportion,
            bottom=bottom,
            color=custom_colours[product_type],
            label=product_type,
            zorder=2
        )
        
        # Create legend
        legend_handles.append(plt.Rectangle((0, 0), 1, 1, color=custom_colours[product_type]))
        legend_labels.append(product_type)
        
        # Handle bar stacking
        if bottom is None:
            bottom = proportion
        else:
            bottom += proportion
        
    # Change figure component colours
    ax.set_facecolor("#f0f0f0")
    ax.grid(zorder=0, linestyle="-", alpha=0.7)
    
    # Arrange legend
    sorted_labels = sorted(legend_labels, reverse=True)
    sorted_handles = [legend_handles[legend_labels.index(label)] for label in sorted_labels]
    
    # Plot legend
    plt.legend(
        sorted_handles,
        sorted_labels,
        title="Animal Product Type",
        title_fontproperties=FontProperties(weight="bold"),
        loc="upper right",
        fontsize=10
    )
    
    # Adjust limits
    plt.ylim(0, 105)
    
    # Add axis components
    plt.ylabel("Percentage (%)", fontsize=14, fontweight="bold")
    plt.xticks()
    
    # Add title
    plt.title(f"IWT Product Type Proportions\nfor {species}", fontsize=16, fontweight="bold")
    plt.tight_layout()
    
    # Add 50% horizontal line
    ax.axhline(y=50, color="#1a1b1c", linestyle="--", linewidth=1, zorder=2)
    
    # Export plot
    plt.savefig(f"Output/{species}.png", dpi=600)
    plt.close()
    #plt.show()